In [1]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
import datetime
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re

data_de_hoje = datetime.date.today()

In [2]:
# Variáveis
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()) )
driver.get("https://mercado.carrefour.com.br/")

time.sleep(1)

precos_carrefour = []


# Busca pelo nome do producto
search = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Pesquise por produtos ou marcas']"))
)
search.click()
search.send_keys("salmão")
search.send_keys(Keys.RETURN)

KeyboardInterrupt: 

In [3]:
search = WebDriverWait(driver, 1).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='open-modal']"))
)
search.click()

# Reire na loja
retire = WebDriverWait(driver, 1).until(
    EC.element_to_be_clickable((By.XPATH, "//button[span[contains(text(), 'Retire na loja')]]"))
)
retire.click()

# Seleciona a cidade (exemplo: primeira cidade válida)
select = Select(WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "select.w-full.border-gray-300.rounded.p-3.mt-4.cursor-pointer.text-sm.drop-shadow-md"))
))

total_cidades = len(select.options) - 1  # Subtrai 1 se quiser ignorar a primeira opção ("Selecionar cidade")
print(f"[INFO] Total de cidades disponíveis: {total_cidades}")



[INFO] Total de cidades disponíveis: 54


# Lista de Cidades e Lojas Coletadas

In [5]:
# Abrir o modal de seleção de cidade
search = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='open-modal']"))
)
search.click()

# Clica em "Retire na loja"
retire = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.XPATH, "//button[span[contains(text(), 'Retire na loja')]]"))
)
retire.click()

# Captura o dropdown de cidades
select = Select(WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "select.w-full.border-gray-300.rounded.p-3.mt-4.cursor-pointer.text-sm.drop-shadow-md"))
))

total_cidades = len(select.options)
print(f"[INFO] Total de cidades disponíveis: {total_cidades}")

# Lista para guardar todas as lojas de todas as cidades
lista_lojas = []

# Loop sobre todas as cidades (ignorando o índice 0 se for "Selecionar cidade")
for cidade_index in range(1, total_cidades):
    try:
        cidade_nome = select.options[cidade_index].text
        print(f"[INFO] Coletando lojas da cidade: {cidade_nome}")

        # Seleciona a cidade
        select.select_by_index(cidade_index)
        time.sleep(3)  # Dá um tempo para o carregamento

        # Captura todas as lojas exibidas
        lojas = driver.find_elements(By.CSS_SELECTOR, "li.border.rounded-lg.shadow-md.p-4")

        for loja in lojas:
            texto_loja = loja.text.strip()
            if texto_loja:
                lista_lojas.append({
                    "cidade": cidade_nome,
                    "loja": texto_loja
                })

    except Exception as e:
        print(f"[WARNING] Erro ao processar cidade {cidade_nome}: {e}")
        continue

# Exibe o total de lojas coletadas
print(f"[INFO] Total de lojas coletadas: {len(lista_lojas)}")

# Converte para DataFrame
df_lojas_carrefour = pd.DataFrame(lista_lojas)


[INFO] Total de cidades disponíveis: 55
[INFO] Coletando lojas da cidade: Maceió - AL
[INFO] Coletando lojas da cidade: Manaus - AM
[INFO] Coletando lojas da cidade: Itabuna - BA
[INFO] Coletando lojas da cidade: Brasília - DF
[INFO] Coletando lojas da cidade: Vila Velha - ES
[INFO] Coletando lojas da cidade: Goiânia - GO
[WARNING] Erro ao processar cidade Goiânia - GO: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=137.0.7151.104)
Stacktrace:
0   chromedriver                        0x0000000103412668 cxxbridge1$str$ptr + 2723108
1   chromedriver                        0x000000010340a8dc cxxbridge1$str$ptr + 2690968
2   chromedriver                        0x0000000102f5e714 cxxbridge1$string$len + 90428
3   chromedriver                        0x0000000102f38740 chromedriver + 132928
4   chromedriver                        0x0000000102fcd9e4 cxxbridge1$string$len + 545804
5   chromedriver                        0x0000